# Heterogeneity Assessments
### Sarah M. McDonald

In [53]:
# imports
import pandas as pd
import geopandas as gpd
import numpy as np
from openpyxl import load_workbook

# paths
folder = r""
hex01_path = f"{folder}/heterogeneity/DC_LC_hex01.dbf"
hex1_path = f"{folder}/heterogeneity/DC_LC_hex1.dbf"
accuracy_table_path = f"{folder}/fuzzy_TA/summary_tables/DC_fuzzy3x3_tables.xlsx"
points_path = f"{folder}/lcc_aa_points_cleaned.gpkg"
hex_grid_paths = f"{folder}/heterogeneity/DC_hexagons.gpkg"
excel_path = f"{folder}/heterogeneity/DC_heterogeneity_accuracy.xlsx"

In [ ]:
# read in hex results and clean up data
def clean_up(path, all_classes:bool=True):
    df = (
        gpd.read_file(path)
        .drop(['geometry'], axis=1)
    )

    if all_classes:
        # compute % per class
        cols = {
            1   : "Water",
            2   : "Wetlands",
            3   : "Tree Canopy",
            4   : "Shrubland",
            5   : "Low Vegetation",
            6   : "Barren",
            7   : "Structures",
            8   : "Other Impervious",
            9   : "Roads",
            10  : "TC Over Structures",
            11  : "TC Over Other Imp",
            12  : "TC Over Roads",
        }

        # compute % class
        df.loc[:, 'total'] = df[[f"VALUE_{x}" for x in cols]].sum(axis=1)
        df = df.query("total > 0")

        for c in cols:
            df.loc[:, f"{cols[c]}"] = df[f"VALUE_{c}"] / df['total']
            df.loc[:, f"H_{c}"] = np.log(df[f"{cols[c]}"]) * df[f"{cols[c]}"]

        # compute shannon's diveristy index
        df.loc[:, 'H'] = df[[f"H_{i}" for i in cols.keys()]].sum(axis=1) * -1

        # calculate shannon's equitability index EH 
        df.loc[:, 'EH'] = df['H'] / np.log(len(cols))

        # col order
        c = ['GRID_ID', 'total'] + list(cols.values()) + ['H', 'EH']
        
    else:

        # compute % per class
        cols = {
            "Water": [1],
            "Herbaceous" : [2,4,5],
            "Tree Canopy" : [3, 10, 11, 12],
            "Barren" : [6],
            "Impervious" : [7, 8, 9],
        }

        # compute % class
        df.loc[:, 'total'] = df[[x for x in list(df) if 'VALUE_' in x]].sum(axis=1)
        df = df.query("total > 0")

        for c in cols:
            df.loc[:, c] = df[[f"VALUE_{i}" for i in cols[c]]].sum(axis=1) / df['total']
            df.loc[:, f"H_{c}"] = np.log(df[c]) * df[c]

        # compute shannon's diveristy index
        df.loc[:, 'H'] = df[[f"H_{i}" for i in cols.keys()]].sum(axis=1) * -1

        # calculate shannon's equitability index EH 
        df.loc[:, 'EH'] = df['H'] / np.log(len(cols))

        # col order
        c = ['GRID_ID', 'total'] + list(cols.keys()) + ['H', 'EH']

    # return data
    return df[c]

# call and print df
hex1 = clean_up(hex1_path)
hex01 = clean_up(hex01_path)
hex1_5 = clean_up(hex1_path, all_classes=False)
hex01_5 = clean_up(hex01_path, all_classes=False)

hex1.to_excel(excel_path, sheet_name="hex1_shannon", index=False)
book = load_workbook(excel_path)
writer = pd.ExcelWriter(excel_path, engine = 'openpyxl')
writer.book = book
hex1_5.to_excel(writer, sheet_name="hex1_LC5_shannon", index=False)
hex01.to_excel(writer, sheet_name="hex01_shannon", index=False)
hex01_5.to_excel(writer, sheet_name="hex01_LC5_shannon", index=False)
writer.close()

# read in points sjoin with grid
points = (
    gpd.read_file(points_path, layer='AA_clean')
    .filter(items=['uid', 'geometry'])
    .rename(columns={'uid':'point_uid'})
)
hex1_gdf = (
    gpd.read_file(hex_grid_paths, layer='hex1')
    .filter(items=['GRID_ID', 'uid', 'geometry'])
)
hex1_sj = (
    gpd.sjoin(hex1_gdf[['GRID_ID','uid','geometry']], points)
    .filter(items=['GRID_ID', 'uid', 'point_uid'])
)
hex01_gdf = (
    gpd.read_file(hex_grid_paths, layer='hex01')
    .filter(items=['GRID_ID', 'uid', 'geometry'])   
)
hex01_sj = (
    gpd.sjoin(hex01_gdf[['GRID_ID', 'uid', 'geometry']], points)
    .filter(items=['GRID_ID', 'uid', 'point_uid'])
)

# merge data to grids 
hex1_gdf = (
    hex1_gdf
    .merge(hex1[['GRID_ID', 'total', 'H', 'EH']], on='GRID_ID', how='left')
    .merge(hex1_5[['GRID_ID', 'H', 'EH']].rename(columns={'H':"H5", 'EH':"EH5"}), on='GRID_ID', how='left')
)

hex01_gdf = (
    hex01_gdf
    .merge(hex01[['GRID_ID', 'total', 'H', 'EH']], on='GRID_ID', how='left')
    .merge(hex01_5[['GRID_ID', 'H', 'EH']].rename(columns={'H':"H5", 'EH':"EH5"}), on='GRID_ID', how='left')
)

# read aa data and merge to points
aa_df = (
    pd.read_excel(accuracy_table_path, sheet_name='all_data')
    .filter(items=['uid', 'GT_static', 'Map_static', 'GT_st_3x3'])
)
aa_df.loc[:, 'GT_st'] = False
aa_df.loc[aa_df['GT_static']==aa_df['Map_static'], 'GT_st'] = True
aa_df = aa_df[['uid', 'GT_st', 'GT_st_3x3']]
points = (
    points
    .merge(aa_df, left_on='point_uid', right_on='uid')
    .drop('uid', axis=1)
    .merge(hex1_sj[['GRID_ID', 'point_uid']].rename(columns={'GRID_ID':'hex1_grid'}), on='point_uid', how='left')
    .merge(hex01_sj[['GRID_ID', 'point_uid']].rename(columns={'GRID_ID':'hex01_grid'}), on='point_uid', how='left')
)
 
# count number of points per grid, number of correct points per grid, and calculate % correct per grid
h1_counts = (
    (
    points
    .groupby('hex1_grid')
    .size()
    .reset_index()
    .rename(columns={0:"hex1_numPts"})
    )
    .merge(
        (
            points
            .query("GT_st")
            .groupby('hex1_grid')
            .size()
            .reset_index()
            .rename(columns={0:"hex1_numPtsT"})
        ),
        on='hex1_grid',
        how='outer'
    )
    .merge(
        (
            points
            .query("GT_st_3x3")
            .groupby('hex1_grid')
            .size()
            .reset_index()
            .rename(columns={0:"hex1_numPtsT3x3"})
        ),
        on='hex1_grid',
        how='outer'
    )
    .fillna(0)
)
h1_counts.loc[:, 'hex1_pcTrue'] = h1_counts['hex1_numPtsT'] / h1_counts['hex1_numPts']
h1_counts.loc[:, 'hex1_pcTrue3x3'] = h1_counts['hex1_numPtsT3x3'] / h1_counts['hex1_numPts']
h1_counts = h1_counts[['hex1_grid', 'hex1_numPts', 'hex1_pcTrue', 'hex1_pcTrue3x3']]
hex1_gdf = (
    hex1_gdf
    .merge(
        h1_counts,
        left_on='GRID_ID',
        right_on='hex1_grid',
        how='left'
    )
)
# hex1_gdf.to_file(hex_grid_paths, layer='hex1', driver="GPKG")

# count number of points per grid, number of correct points per grid, and calculate % correct per grid
h01_counts = (
    (
    points
    .groupby('hex01_grid')
    .size()
    .reset_index()
    .rename(columns={0:"hex01_numPts"})
    )
    .merge(
        (
            points
            .query("GT_st")
            .groupby('hex01_grid')
            .size()
            .reset_index()
            .rename(columns={0:"hex01_numPtsT"})
        ),
        on='hex01_grid',
        how='outer'
    )
    .merge(
        (
            points
            .query("GT_st_3x3")
            .groupby('hex01_grid')
            .size()
            .reset_index()
            .rename(columns={0:"hex01_numPtsT3x3"})
        ),
        on='hex01_grid',
        how='outer'
    )
    .fillna(0)
)
h01_counts.loc[:, 'hex01_pcTrue'] = h01_counts['hex01_numPtsT'] / h01_counts['hex01_numPts']
h01_counts.loc[:, 'hex01_pcTrue3x3'] = h01_counts['hex01_numPtsT3x3'] / h01_counts['hex01_numPts']
h01_counts = h01_counts[['hex01_grid', 'hex01_numPts', 'hex01_pcTrue', 'hex01_pcTrue3x3']]
hex01_gdf = (
    hex01_gdf
    .merge(
        h01_counts,
        left_on='GRID_ID',
        right_on='hex01_grid',
        how='left'
    )
)
# hex01_gdf.to_file(hex_grid_paths, layer='hex01', driver="GPKG")
points = (
    points
    .merge(
        (
            hex1_gdf
            .filter(items=['GRID_ID', 'H', 'EH', 'H5', 'EH5'], axis=1)
            .rename(columns={'H':'H_hex1', 'EH':'EH_hex1', 'H5':'H5_hex1', 'EH5':'EH5_hex1'})
        ), 
        left_on='hex1_grid', 
        right_on='GRID_ID'
    )
    .drop('GRID_ID', axis=1)
    .merge(
        (
            hex01_gdf
            .filter(items=['GRID_ID', 'H', 'EH', 'H5', 'EH5'], axis=1)
            .rename(columns={'H':'H_hex01', 'EH':'EH_hex01', 'H5':'H5_hex01', 'EH5':'EH5_hex01'})
        ), 
        left_on='hex01_grid', 
        right_on='GRID_ID'
    )
    .drop(['GRID_ID', 'geometry'], axis=1)
)

book = load_workbook(excel_path)
writer = pd.ExcelWriter(excel_path, engine = 'openpyxl')
writer.book = book
points.to_excel(writer, sheet_name="all_data", index=False)
hex1_gdf.drop('geometry', axis=1).to_excel(writer, sheet_name="hex1_summary", index=False)
hex01_gdf.drop('geometry', axis=1).to_excel(writer, sheet_name="hex01_summary", index=False)
writer.close()

In [ ]:
import os
import pandas as pd
import geopandas as gpd

# ta files
ta_folder = f"{folder}/heterogeneity/RegionGroup/TA"
ta_files = os.listdir(ta_folder)

# iterate files paths, read them, and aggregate over hex id and region group values
data = {}
for ta in ta_files:
    data[ta] = (
        pd.read_csv(f"{ta_folder}/{ta}")
        .groupby(['uid', 'RegionID'])
        .sum()
        .reset_index()
        .query("uid > 0")
        .query("RegionID > 0")
    )

# land cover names per value
lc = {
    1   : "Water",
    2   : "Wetlands",
    3   : "Tree Canopy",
    4   : "Shrubland",
    5   : "Low Vegetation",
    6   : "Barren",
    7   : "Structures",
    8   : "Other Impervious",
    9   : "Roads",
    10  : "TC Over Structures",
    11  : "TC Over Other Impervious",
    12  : "TC Over Roads",
}

lc5 = {
    1   : "Water",
    3   : "Tree Canopy",
    4   : "Herbaceous",
    6   : "Barren",
    7   : "Impervious",
}

# read in region group crosswalk to id/lc class
cw_lc = (
    gpd.read_file(f"{folder}/heterogeneity/RegionGroup/DC_LC_regionGroup8.tif.vat.dbf")
    .filter(items=['Value', 'LINK'], axis=1)
    .rename(columns={'Value':'RegionID', 'LINK':'LC_val'})
)
for i in lc:
    cw_lc.loc[cw_lc['LC_val'] == i, "LC"] = lc[i]

cw_lc5 = (
    gpd.read_file(f"{folder}/heterogeneity/RegionGroup/DC_LC_5class_regionGroup8.tif.vat.dbf")
    .filter(items=['Value', 'LINK'], axis=1)
    .rename(columns={'Value':'RegionID', 'LINK':'LC_val'})
)
for i in lc5:
    cw_lc5.loc[cw_lc5['LC_val'] == i, "LC"] = lc5[i]

# merge land cover to ta results
book = load_workbook(excel_path)
writer = pd.ExcelWriter(excel_path, engine = 'openpyxl')
writer.book = book
for ta in data:
    if "5class" in ta:
        data[ta] = (
            data[ta]
            .merge(cw_lc5, on='RegionID', how='left')
        )
    else:
        data[ta] = (
            data[ta]
            .merge(cw_lc, on='RegionID', how='left')
        )
    data[ta].to_excel(writer, sheet_name=ta, index=False)
writer.close()